In [30]:
import pandas as pd
import os
data_folder = "../data"

In [31]:
hospitals = pd.read_csv(os.path.join(data_folder, 'opportunity', 'hospitals.csv'))
supermarkets = pd.read_csv(os.path.join(data_folder, 'opportunity', 'supermarkets.csv'))
daycares = pd.read_csv(os.path.join(data_folder, 'opportunity', 'daycares.csv'))
census = pd.read_csv(os.path.join(data_folder, 'census', 'hex_census.csv'))
runs = pd.read_csv(os.path.join(data_folder, 'run_reference.csv'))
census.head()

,id,pop_2016,age_total,age_0_14,age_65_plus,lone_total,lang_total,lang_french,lang_neither,inc_total,...,imm_recent,ind_total,ind_ind,house_total,house_30p_shelter,tenant_total,tenant_30p_shelter,lab_total,lab_inforce,lab_unemp
0,24,0.015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,129,0.025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,130,0.100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,131,0.020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,132,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
# Calculate the access scores
for idx, row in runs.iterrows():
    print("Running matrix", row['matrix_id'])
    ttm = pd.read_csv(os.path.join(data_folder, 'matrices', f"{row['matrix_id']}_{row['gtfs_feed']}.csv"))
    ttm.travel_time = ttm.travel_time.fillna(999)
    # Nearest Hospital
    df = pd.merge(ttm, hospitals, left_on='to_id', right_on='id')
    df = df[df.hospitals > 0]
    nearest_hospital = df[['from_id', 'travel_time']].groupby('from_id', as_index=False).min()
    
    # Nearest Supermarket
    df = pd.merge(ttm, supermarkets, left_on='to_id', right_on='id')
    df = df[df.supermarkets > 0]
    nearest_supermarket = df[['from_id', 'travel_time']].groupby('from_id', as_index=False).min()
    
    # Cumulative Daycare Spaces
    df = pd.merge(ttm, daycares, left_on='to_id', right_on='id')
    df = df[df.travel_time <= 45]
    daycare_spots = df[['from_id', 'daycare_spots']].groupby('from_id', as_index=False).sum()
    
    # Cumulative Population Access
    df = pd.merge(ttm, census[['id', 'pop_2016']], left_on='to_id', right_on='id')
    df = df[df.travel_time <= 45]
    population = df[['from_id', 'pop_2016']].groupby('from_id', as_index=False).sum()
    
    final = pd.merge(nearest_hospital, nearest_supermarket, on='from_id')
    final = pd.merge(final, daycare_spots, on='from_id')
    final = pd.merge(final, population, on='from_id')
    final.columns = ['id', 'nearest_hospital', 'nearest_supermarket', 'daycare_spots', 'population']
    final.to_csv(os.path.join(data_folder, 'results', f"{row['matrix_id']}_results.csv"), index=False)

Running matrix 1
Running matrix 2
Running matrix 3
Running matrix 4
Running matrix 5
Running matrix 6
Running matrix 7
Running matrix 8
Running matrix 9
Running matrix 10
Running matrix 11
Running matrix 12
Running matrix 13
Running matrix 14
Running matrix 15
Running matrix 16
Running matrix 17
Running matrix 18
Running matrix 19
Running matrix 20
Running matrix 21
Running matrix 22
Running matrix 23
Running matrix 24
Running matrix 25
Running matrix 26
Running matrix 27
